###### Import Statement

In [305]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pydot
import copy
import random
from random import randrange
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score,KFold
from sklearn.tree import export_graphviz
from sklearn.preprocessing import LabelEncoder
from scipy import misc
from collections import OrderedDict
from operator import itemgetter
%matplotlib inline


###### Preprocessing

In [306]:
#Titanic Dataset
#https://www.kaggle.com/c/titanic/data
def get_titanic_data():
    df_titanic=pd.read_csv('titanic.csv',)
    #Remove unnecessary columns (body= Body Indentification number, Name= Name)
    df_titanic.drop(['name','body'],1,inplace=True)
    #Fill all the na  
    df_titanic.cabin.fillna('unknown',inplace=True)
    df_titanic.age.fillna(df_titanic['age'].mean(),inplace=True)
    df_titanic.fillna(0,inplace=True)
    #Covert nonnumeric value into numeric
    df_titanic['sex'] = LabelEncoder().fit_transform(df_titanic['sex'])
    df_titanic['cabin'] = LabelEncoder().fit_transform(df_titanic['cabin'].astype(str))
    df_titanic['embarked'] = LabelEncoder().fit_transform(df_titanic['embarked'].astype(str))
    df_titanic['home.dest'] = LabelEncoder().fit_transform(df_titanic['home.dest'].astype(str))
    df_titanic['ticket'] = LabelEncoder().fit_transform(df_titanic['ticket'])
    df_titanic['boat'] = LabelEncoder().fit_transform(df_titanic['boat'].astype(str))
    # df_titanic.head()
    # df_titanic.dtypes
    # print(df_titanic.isnull().sum())
    y = df_titanic['pclass']
    X = df_titanic.drop("pclass", axis = 1)
    return X,y

###### Train_Test_Spliting

In [307]:
#To split the dataset into 3 parts
def data_split(X_Data,y_Target):   
    X_train, X_test, y_train, y_test= train_test_split(X_Data, y_Target, test_size=0.4)
    X_train, X_val, y_train, y_val= train_test_split(X_train, y_train, test_size=0.5)
    return X_train,X_val,X_test,y_train,y_val,y_test

In [308]:
X,y=get_titanic_data()
X_train,X_val,X_test,y_train,y_val,y_test=data_split(X,y)

###### Methods and submethods declaration for genetic algorithm optimization

In [309]:
#Method used to split the initial random forest into random forest vector population
def partition_gene_to_chromosome(listin, n):
    random.shuffle(listin)
    return [listin[i::n] for i in range(n)]

In [310]:
#Function to determin the best fit parents
def call_population_fitness(randomForest_population):
    print(len(randomForest_population))
    fitnesslist=[]
    for rf in randomForest_population:
        fitness=rf.score(X_val,y_val)
        fitnesslist.append(fitness)
    fitness_matrix = { i : fitnesslist[i] for i in range(0, len(fitnesslist) ) }
    print(fitness_matrix)
    return fitness_matrix

In [311]:
#Fuction to select the parent population
def call_population_selection(fitness_matrix,randomForest_population,parents_number):
    fitness_ordered=sorted(fitness_matrix.items(), key=itemgetter(1),reverse=True)
    selected_chromRF_Index=list(list(zip(*fitness_ordered))[0])[:parents_number]
#     print(selected_chromRF_Index)
    selected_rf_population=[randomForest_population[i] for i in selected_chromRF_Index]
#     print("Selection",selected_rf_population)
    return selected_rf_population

In [312]:
#Sub Function for cross over
def get_offcross_spring(rf_chromosome1,rf_chromosome2,cross_over_point):
#     print("\n\n\n\n\n\n\n\n\n\n\n\*****************************\n\n\n\n\n")
#     for i in range(len(rf_chromosome2.estimators_)):
#         print(rf_chromosome2.estimators_[i].random_state)
#     print(rf_chromosome2.estimators_,len(rf_chromosome2.estimators_),"minuuuuuuuuuuuuuuuuroseeeeeeeeeeeeeeeeeeeee")
#     print("\n\n\n\n\n\n\n\n\n\n\n\*****************************\n\n\n\n\n")
#     for i in range(len(rf_chromosome1.estimators_)):
#         print(rf_chromosome1.estimators_[i].random_state)
#     print(rf_chromosome1.estimators_,len(rf_chromosome1.estimators_),"fousiyyaaaasaleeeeemmmmmmm")
#     print("\n\n\n\n\n\n\n\n\n\n\n\*****************************\n\n\n\n\n")
#     print(cross_over_point)
    cross_1=rf_chromosome1.estimators_[0:cross_over_point]
#     print("\n\n\n\n\n\n\n\n\n\n\n\*****************************\n\n\n\n\n")
#     print(cross_1,len(cross_1),"fousiyyaaaa")
    cross_2=rf_chromosome2.estimators_[cross_over_point:]
#     print("\n\n\n\n\n\n\n\n\n\n\n\*****************************\n\n\n\n\n")
#     print(cross_2,len(cross_2),"minuuuuuuuuuuuuuuuu")
    modified_estimators=cross_1+cross_2
#     print("\n\n\n\n\n\n\n\n\n\n\n\*****************************\n\n\n\n\n")
#     for i in range(len(modified_estimators)):
#         print(modified_estimators[i].random_state)
#     print(modified_estimators,"anwarrrrrrrrrrr")
    rf_chromosome1.estimators_=modified_estimators
#     print(rf_chromosome1)
    return rf_chromosome1

In [313]:
# Main Crossover Function
def call_crossover_generation(selected_rf_population,crossover_per_gen):
#     print("\n\n\n\n\n\n\n\n\n\n\n\*****************************\n\n\n\n\n")
#     print(selected_rf_population[1].estimators_)
#     print("\n\n\n\n\n\n\n\n\n\n\n\*****************************\n\n\n\n\n")
    offsprings_gen = []
    # The crossoverpoint
    cross_over_point = np.uint8(len(selected_rf_population[1])/2)   
    for k in range(int(crossover_per_gen)):
        # Crossoverindex of the first parent
        parent1_index = k % len(selected_rf_population)
        # Crossoverindex of the seconf parent
        parent2_index = (k+1) % len(selected_rf_population)
        offspring=get_offcross_spring(selected_rf_population[parent1_index],selected_rf_population[parent2_index],cross_over_point)
        offsprings_gen.append(offspring)
#         print(k)
#         print(offspring.estimators_)
#     print(len(offsprings_gen))
    return offsprings_gen

In [314]:
# Method to get the mutated chromosomes
def get_mutation_gen(randomForest,selected_rf_population,mutation_per_gen):
    offsprings_gen = []
    rf_for_mutation=[]
    for k in range(int(mutation_per_gen)):  
        # Randomly choose a chromosome for mutation.
        parent1_index = k % len(selected_rf_population)
        rf_for_mutation=random.choice(selected_rf_population)
        #Randomly choose one of the gene for mutation
        gene_for_mutation = random.choice(rf_for_mutation.estimators_)
        rf_for_mutation.estimators_.remove(gene_for_mutation)
        slct_impurity_df=random.choice(randomForest)
        rf_for_mutation.estimators_.append(slct_impurity_df)
        offsprings_gen.append(rf_for_mutation)
#     print(offsprings_gen[0].estimators_)
    return offsprings_gen

In [319]:
M,N=X.shape
F=int(N**(1/2))

#Total number of decision trees all random forest
initial_population=1000
initial_population_list=list(range(0,initial_population))

#Total number of chromosome(No of sub random forest for genetic algorithm)
n=100
cross_over_rate=0.9
mutation_rate=0.1
number_of_gen=100
crossover_per_gen=(cross_over_rate*n)
mutation_per_gen=(mutation_rate*n)
print(crossover_per_gen)
print(mutation_per_gen)

#Number of parents for next generation
parents_number=5

# Random forest model 
# Generating first population of raandom forest
randomForest = RandomForestClassifier(n_estimators=initial_population,max_features=F)
randomForest.fit(X_train,y_train)
print(len(randomForest.estimators_))

90.0
10.0
1000


In [320]:
sub_forest_chromosome=partition_gene_to_chromosome(initial_population_list,n)

In [321]:
#List of initial population (chromosomes)
randomForest_population=[]
for chrom in range(n):
    rf_classifier=RandomForestClassifier(n_estimators=len(sub_forest_chromosome[chrom]))
    rf_classifier.estimators_=[]
    for gene in sub_forest_chromosome[chrom]:
        rf_classifier.estimators_.append(randomForest.estimators_[(gene-1)])    
    rf_classifier.classes_=randomForest.classes_
    rf_classifier.n_classes_=randomForest.n_classes_
    rf_classifier.n_outputs_=randomForest.n_outputs_ 
    randomForest_population.append(rf_classifier)


In [322]:
for gen in range(number_of_gen):
    print("Generation Number: ", gen)
    # To measure the fitness of each chromosome
    fitness_matrix=call_population_fitness(randomForest_population)

    # To select the best parents for next generation
    # print("next_gen_parents\n\n\n\n\n\n\n\n\n\n",randomForest_population)
    # print("next_gen_parents decision tree                 \n\n\n\n\n\n\n\n\n\n",randomForest_population[0].estimators_)
    next_gen_parents = call_population_selection(fitness_matrix,randomForest_population,parents_number)

    # Crossover for generating offspring
    # print("crossover_offsprings\n\n\n\n\n\n\n\n\n\n",next_gen_parents)
    # print("crossover_offsprings decision tree\n\n\n\n\n\n\n\n\n\n",next_gen_parents[0].estimators_)
    crossover_offsprings = call_crossover_generation(copy.deepcopy(next_gen_parents),crossover_per_gen)
    # print(crossover_offsprings)
    # Mutation for the generated offspring
    # print(next_gen_parents)
    mutated_offspring = get_mutation_gen(randomForest,copy.deepcopy(next_gen_parents),mutation_per_gen)
    # print(mutated_offspring.estimators_)
    randomForest_population=[]
    randomForest_population=mutated_offspring+crossover_offsprings
#     print(randomForest_population[1].estimators_)
    #     # New population for next generation
    #     next_population=[]
    #     next_population.append(crossover_offsprings)
    #     next_population.append(mutated_offspring)
    #     print(len(next_population))
    #     randomForest_population=next_population


Generation Number:  0
100
{0: 0.9669211195928753, 1: 0.9796437659033079, 2: 0.9694656488549618, 3: 0.9669211195928753, 4: 0.9414758269720102, 5: 0.9541984732824428, 6: 0.9567430025445293, 7: 0.9541984732824428, 8: 0.9618320610687023, 9: 0.9592875318066157, 10: 0.9592875318066157, 11: 0.9567430025445293, 12: 0.9541984732824428, 13: 0.9567430025445293, 14: 0.9643765903307888, 15: 0.9745547073791349, 16: 0.9669211195928753, 17: 0.9694656488549618, 18: 0.9694656488549618, 19: 0.9491094147582697, 20: 0.9618320610687023, 21: 0.9541984732824428, 22: 0.9720101781170484, 23: 0.9669211195928753, 24: 0.9491094147582697, 25: 0.9618320610687023, 26: 0.9720101781170484, 27: 0.9618320610687023, 28: 0.9491094147582697, 29: 0.9694656488549618, 30: 0.9745547073791349, 31: 0.9618320610687023, 32: 0.9643765903307888, 33: 0.9541984732824428, 34: 0.9643765903307888, 35: 0.9516539440203562, 36: 0.9720101781170484, 37: 0.9465648854961832, 38: 0.9516539440203562, 39: 0.9669211195928753, 40: 0.9745547073791349,

{0: 0.9669211195928753, 1: 0.9669211195928753, 2: 0.9669211195928753, 3: 0.9669211195928753, 4: 0.9669211195928753, 5: 0.9669211195928753, 6: 0.9669211195928753, 7: 0.9669211195928753, 8: 0.9669211195928753, 9: 0.9669211195928753, 10: 0.9796437659033079, 11: 0.9796437659033079, 12: 0.9796437659033079, 13: 0.9796437659033079, 14: 0.9796437659033079, 15: 0.9796437659033079, 16: 0.9796437659033079, 17: 0.9796437659033079, 18: 0.9796437659033079, 19: 0.9796437659033079, 20: 0.9796437659033079, 21: 0.9796437659033079, 22: 0.9796437659033079, 23: 0.9796437659033079, 24: 0.9796437659033079, 25: 0.9796437659033079, 26: 0.9796437659033079, 27: 0.9796437659033079, 28: 0.9796437659033079, 29: 0.9796437659033079, 30: 0.9796437659033079, 31: 0.9796437659033079, 32: 0.9796437659033079, 33: 0.9796437659033079, 34: 0.9796437659033079, 35: 0.9796437659033079, 36: 0.9796437659033079, 37: 0.9796437659033079, 38: 0.9796437659033079, 39: 0.9796437659033079, 40: 0.9796437659033079, 41: 0.9796437659033079, 4

{0: 0.9745547073791349, 1: 0.9745547073791349, 2: 0.9745547073791349, 3: 0.9745547073791349, 4: 0.9745547073791349, 5: 0.9745547073791349, 6: 0.9745547073791349, 7: 0.9745547073791349, 8: 0.9745547073791349, 9: 0.9745547073791349, 10: 0.9796437659033079, 11: 0.9796437659033079, 12: 0.9796437659033079, 13: 0.9796437659033079, 14: 0.9796437659033079, 15: 0.9796437659033079, 16: 0.9796437659033079, 17: 0.9796437659033079, 18: 0.9796437659033079, 19: 0.9796437659033079, 20: 0.9796437659033079, 21: 0.9796437659033079, 22: 0.9796437659033079, 23: 0.9796437659033079, 24: 0.9796437659033079, 25: 0.9796437659033079, 26: 0.9796437659033079, 27: 0.9796437659033079, 28: 0.9796437659033079, 29: 0.9796437659033079, 30: 0.9796437659033079, 31: 0.9796437659033079, 32: 0.9796437659033079, 33: 0.9796437659033079, 34: 0.9796437659033079, 35: 0.9796437659033079, 36: 0.9796437659033079, 37: 0.9796437659033079, 38: 0.9796437659033079, 39: 0.9796437659033079, 40: 0.9796437659033079, 41: 0.9796437659033079, 4

{0: 0.9541984732824428, 1: 0.9541984732824428, 2: 0.9541984732824428, 3: 0.9541984732824428, 4: 0.9541984732824428, 5: 0.9541984732824428, 6: 0.9541984732824428, 7: 0.9541984732824428, 8: 0.9541984732824428, 9: 0.9541984732824428, 10: 0.9796437659033079, 11: 0.9796437659033079, 12: 0.9796437659033079, 13: 0.9796437659033079, 14: 0.9796437659033079, 15: 0.9796437659033079, 16: 0.9796437659033079, 17: 0.9796437659033079, 18: 0.9796437659033079, 19: 0.9796437659033079, 20: 0.9796437659033079, 21: 0.9796437659033079, 22: 0.9796437659033079, 23: 0.9796437659033079, 24: 0.9796437659033079, 25: 0.9796437659033079, 26: 0.9796437659033079, 27: 0.9796437659033079, 28: 0.9796437659033079, 29: 0.9796437659033079, 30: 0.9796437659033079, 31: 0.9796437659033079, 32: 0.9796437659033079, 33: 0.9796437659033079, 34: 0.9796437659033079, 35: 0.9796437659033079, 36: 0.9796437659033079, 37: 0.9796437659033079, 38: 0.9796437659033079, 39: 0.9796437659033079, 40: 0.9796437659033079, 41: 0.9796437659033079, 4

{0: 0.9541984732824428, 1: 0.9541984732824428, 2: 0.9541984732824428, 3: 0.9541984732824428, 4: 0.9541984732824428, 5: 0.9541984732824428, 6: 0.9541984732824428, 7: 0.9541984732824428, 8: 0.9541984732824428, 9: 0.9541984732824428, 10: 0.9796437659033079, 11: 0.9796437659033079, 12: 0.9796437659033079, 13: 0.9796437659033079, 14: 0.9796437659033079, 15: 0.9796437659033079, 16: 0.9796437659033079, 17: 0.9796437659033079, 18: 0.9796437659033079, 19: 0.9796437659033079, 20: 0.9796437659033079, 21: 0.9796437659033079, 22: 0.9796437659033079, 23: 0.9796437659033079, 24: 0.9796437659033079, 25: 0.9796437659033079, 26: 0.9796437659033079, 27: 0.9796437659033079, 28: 0.9796437659033079, 29: 0.9796437659033079, 30: 0.9796437659033079, 31: 0.9796437659033079, 32: 0.9796437659033079, 33: 0.9796437659033079, 34: 0.9796437659033079, 35: 0.9796437659033079, 36: 0.9796437659033079, 37: 0.9796437659033079, 38: 0.9796437659033079, 39: 0.9796437659033079, 40: 0.9796437659033079, 41: 0.9796437659033079, 4

...

{0: 0.9541984732824428, 1: 0.9541984732824428, 2: 0.9541984732824428, 3: 0.9541984732824428, 4: 0.9541984732824428, 5: 0.9541984732824428, 6: 0.9541984732824428, 7: 0.9541984732824428, 8: 0.9541984732824428, 9: 0.9541984732824428, 10: 0.9821882951653944, 11: 0.9821882951653944, 12: 0.9821882951653944, 13: 0.9821882951653944, 14: 0.9821882951653944, 15: 0.9821882951653944, 16: 0.9821882951653944, 17: 0.9821882951653944, 18: 0.9821882951653944, 19: 0.9821882951653944, 20: 0.9821882951653944, 21: 0.9821882951653944, 22: 0.9821882951653944, 23: 0.9821882951653944, 24: 0.9821882951653944, 25: 0.9821882951653944, 26: 0.9821882951653944, 27: 0.9821882951653944, 28: 0.9821882951653944, 29: 0.9821882951653944, 30: 0.9821882951653944, 31: 0.9821882951653944, 32: 0.9821882951653944, 33: 0.9821882951653944, 34: 0.9821882951653944, 35: 0.9821882951653944, 36: 0.9821882951653944, 37: 0.9821882951653944, 38: 0.9821882951653944, 39: 0.9821882951653944, 40: 0.9821882951653944, 41: 0.9821882951653944, 4

{0: 0.9669211195928753, 1: 0.9669211195928753, 2: 0.9669211195928753, 3: 0.9669211195928753, 4: 0.9669211195928753, 5: 0.9669211195928753, 6: 0.9669211195928753, 7: 0.9669211195928753, 8: 0.9669211195928753, 9: 0.9669211195928753, 10: 0.9847328244274809, 11: 0.9847328244274809, 12: 0.9847328244274809, 13: 0.9847328244274809, 14: 0.9847328244274809, 15: 0.9847328244274809, 16: 0.9847328244274809, 17: 0.9847328244274809, 18: 0.9847328244274809, 19: 0.9847328244274809, 20: 0.9847328244274809, 21: 0.9847328244274809, 22: 0.9847328244274809, 23: 0.9847328244274809, 24: 0.9847328244274809, 25: 0.9847328244274809, 26: 0.9847328244274809, 27: 0.9847328244274809, 28: 0.9847328244274809, 29: 0.9847328244274809, 30: 0.9847328244274809, 31: 0.9847328244274809, 32: 0.9847328244274809, 33: 0.9847328244274809, 34: 0.9847328244274809, 35: 0.9847328244274809, 36: 0.9847328244274809, 37: 0.9847328244274809, 38: 0.9847328244274809, 39: 0.9847328244274809, 40: 0.9847328244274809, 41: 0.9847328244274809, 4

Generation Number:  88
100
{0: 0.9694656488549618, 1: 0.9694656488549618, 2: 0.9694656488549618, 3: 0.9694656488549618, 4: 0.9694656488549618, 5: 0.9694656488549618, 6: 0.9694656488549618, 7: 0.9694656488549618, 8: 0.9694656488549618, 9: 0.9694656488549618, 10: 0.9847328244274809, 11: 0.9847328244274809, 12: 0.9847328244274809, 13: 0.9847328244274809, 14: 0.9847328244274809, 15: 0.9847328244274809, 16: 0.9847328244274809, 17: 0.9847328244274809, 18: 0.9847328244274809, 19: 0.9847328244274809, 20: 0.9847328244274809, 21: 0.9847328244274809, 22: 0.9847328244274809, 23: 0.9847328244274809, 24: 0.9847328244274809, 25: 0.9847328244274809, 26: 0.9847328244274809, 27: 0.9847328244274809, 28: 0.9847328244274809, 29: 0.9847328244274809, 30: 0.9847328244274809, 31: 0.9847328244274809, 32: 0.9847328244274809, 33: 0.9847328244274809, 34: 0.9847328244274809, 35: 0.9847328244274809, 36: 0.9847328244274809, 37: 0.9847328244274809, 38: 0.9847328244274809, 39: 0.9847328244274809, 40: 0.9847328244274809

Generation Number:  92
100
{0: 0.9796437659033079, 1: 0.9796437659033079, 2: 0.9796437659033079, 3: 0.9796437659033079, 4: 0.9796437659033079, 5: 0.9796437659033079, 6: 0.9796437659033079, 7: 0.9796437659033079, 8: 0.9796437659033079, 9: 0.9796437659033079, 10: 0.9847328244274809, 11: 0.9847328244274809, 12: 0.9847328244274809, 13: 0.9847328244274809, 14: 0.9847328244274809, 15: 0.9847328244274809, 16: 0.9847328244274809, 17: 0.9847328244274809, 18: 0.9847328244274809, 19: 0.9847328244274809, 20: 0.9847328244274809, 21: 0.9847328244274809, 22: 0.9847328244274809, 23: 0.9847328244274809, 24: 0.9847328244274809, 25: 0.9847328244274809, 26: 0.9847328244274809, 27: 0.9847328244274809, 28: 0.9847328244274809, 29: 0.9847328244274809, 30: 0.9847328244274809, 31: 0.9847328244274809, 32: 0.9847328244274809, 33: 0.9847328244274809, 34: 0.9847328244274809, 35: 0.9847328244274809, 36: 0.9847328244274809, 37: 0.9847328244274809, 38: 0.9847328244274809, 39: 0.9847328244274809, 40: 0.9847328244274809

Generation Number:  96
100
{0: 0.9694656488549618, 1: 0.9694656488549618, 2: 0.9694656488549618, 3: 0.9694656488549618, 4: 0.9694656488549618, 5: 0.9694656488549618, 6: 0.9694656488549618, 7: 0.9694656488549618, 8: 0.9694656488549618, 9: 0.9694656488549618, 10: 0.9847328244274809, 11: 0.9847328244274809, 12: 0.9847328244274809, 13: 0.9847328244274809, 14: 0.9847328244274809, 15: 0.9847328244274809, 16: 0.9847328244274809, 17: 0.9847328244274809, 18: 0.9847328244274809, 19: 0.9847328244274809, 20: 0.9847328244274809, 21: 0.9847328244274809, 22: 0.9847328244274809, 23: 0.9847328244274809, 24: 0.9847328244274809, 25: 0.9847328244274809, 26: 0.9847328244274809, 27: 0.9847328244274809, 28: 0.9847328244274809, 29: 0.9847328244274809, 30: 0.9847328244274809, 31: 0.9847328244274809, 32: 0.9847328244274809, 33: 0.9847328244274809, 34: 0.9847328244274809, 35: 0.9847328244274809, 36: 0.9847328244274809, 37: 0.9847328244274809, 38: 0.9847328244274809, 39: 0.9847328244274809, 40: 0.9847328244274809